In [2]:
import numpy as np
import matplotlib as plt
import IPythonMagic
from Timer import Timer
import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

In [3]:
%setup_logging

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


In [4]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 11370 / 11441 MB available
Created context handle <53346064>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Michele_Pellegrino/MilanoGPU2018/notebooks/cuda_cache


In [5]:
kernel_src = """

// OH NOUS! THIS ONE DOES NOT WORK!

__global__ void shmemReduction(float* output, float* input, int size, int _nt) {
    
    // Global memory -> maximum foe every thread
    // Block idx is always 0, since we use only one block
    
    int gid = blockIdx.x * blockIdx.x + threadIdx.x;
    float max_value = -9999999.99;
    // float max_value = 0.0;
    for (int i = threadIdx.x; i<size; i = i + blockDim.x) {
        max_value = fmax(max_value, input[i]);
    }
    
    // Check
    output[threadIdx.x] = max_value;
    
    // Find and store the local maximum in shared-memory
    int nt = _nt;
    __shared__ float* max_shared;
    if (threadIdx.x==0) {
        *max_shared = nt;
    }
    __syncthreads();
    max_shared[threadIdx.x] = max_value;
    
    // Sync
    __syncthreads();
    
    // Find the max
    /*
    if (threadIdx.x<32)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+32] );
    if (threadIdx.x<16)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+16] );
    if (threadIdx.x<8)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+8] );
    if (threadIdx.x<4)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+4] );
    if (threadIdx.x<2)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+2] );
    if (threadIdx.x<1)
        max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+1] );
    */
    while ( nt>0 ) {
        if (threadIdx.x<nt)
            max_shared[threadIdx.x] = fmaxf( max_shared[threadIdx.x], max_shared[threadIdx.x+nt] );
        if ( nt>32 )
            __syncthreads();
        nt = nt/2;
    }
    
    // Write to output
    if (threadIdx.x==0)
        output[0] = max_shared[0];
    
}

"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(23): warning: variable "max_shared" is used before its value is set

kernel.cu(9): warning: variable "gid" was declared but never referenced




In [6]:
n = 128
a = np.random.rand(n).astype(np.float32)
print(a)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 64
b = np.empty((1,num_threads)).astype(np.float32)
b_g = GPUArray(b.shape, b.dtype)

[2.19247833e-01 2.09652722e-01 3.77405196e-01 9.46727931e-01
 7.54551649e-01 3.37600887e-01 8.47209513e-01 1.10091055e-02
 1.99576601e-01 5.18888593e-01 9.21331167e-01 9.78999078e-01
 5.16635776e-01 9.01036322e-01 2.12440133e-01 1.97986037e-01
 5.19733965e-01 7.15089858e-01 8.98318589e-02 3.88168454e-01
 5.18138468e-01 8.36922050e-01 3.53600651e-01 4.46089745e-01
 6.83903024e-02 1.73523828e-01 1.09058112e-01 2.50061393e-01
 1.47166908e-01 9.29702818e-01 4.83856313e-02 6.63865581e-02
 6.46059632e-01 9.00658250e-01 1.88357178e-02 3.54131401e-01
 7.29642332e-01 6.25991762e-01 3.40741217e-01 6.60854638e-01
 1.81026578e-01 1.66939288e-01 6.56038523e-01 4.94452983e-01
 9.66727793e-01 8.76094580e-01 6.33156240e-01 6.75236523e-01
 6.75791383e-01 5.70015132e-01 4.79828715e-01 2.17579126e-01
 1.11496694e-01 1.56065792e-01 5.74997842e-01 8.85223821e-02
 4.06107962e-01 9.99683976e-01 2.05004990e-01 9.91612256e-01
 7.90132582e-01 1.96737513e-01 3.29242676e-01 1.08444631e-01
 1.96626022e-01 5.977244

In [7]:
block_size = (num_threads, 1, 1)
grid_size = (1,1,1)

kernel_function(b_g, a_g, np.int32(n), grid=grid_size, block=block_size)

b_g.get(b)
print(a)
print(b)
print(np.max(a))

Exception caught: Resetting to CUDA context context
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-fd1260758ea0>", line 6, in <module>
    b_g.get(b)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/pycuda/gpuarray.py", line 305, in get
    _memcpy_discontig(ary, self, async_=async_, stream=stream)
  File "/home/ubuntu/.local/lib/python3.6/site-packages/pycuda/gpuarray.py", line 1309, in _memcpy_discontig
    drv.memcpy_dtoh(dst, src.gpudata)
pycuda._driver.LogicError: cuMemcpyDtoH failed: an illegal memory access was encountered
Popping <53346064>
Pushing <53346064>


---------------------------------------------------------------------------
LogicError                                Traceback (most recent call last)
~/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py in run_code(self, code_obj, result, async_)
   3264                 else:
-> 3265                     exec(code_obj, self.user_global_ns, self.user_ns)
   3266             finally:

<ipython-input-7-fd1260758ea0> in <module>
      5 
----> 6 b_g.get(b)
      7 print(a)

~/.local/lib/python3.6/site-packages/pycuda/gpuarray.py in get(self, ary, pagelocked, async_, stream, **kwargs)
    304         if self.size:
--> 305             _memcpy_discontig(ary, self, async_=async_, stream=stream)
    306         return ary

~/.local/lib/python3.6/site-packages/pycuda/gpuarray.py in _memcpy_discontig(dst, src, async_, stream)
   1308                 else:
-> 1309                     drv.memcpy_dtoh(dst, src.gpudata)
   1310         else:

LogicError: cuMemcpyDtoH failed: an ille

Custom TB Handler failed, unregistering
